In [1]:
import pandas as pd
import requests

In [2]:
#api_key = 'fK3eae9BKJ8nZKxDTxtAPEXjfi02c3oAhK5aPCdx' #sahana's key
#api_key = 'eNxKT9YQ2JbnD6J2TjemGwcGDOfdKwLutysmZ7YM' #madelyn's key
base_url = 'http://api.data.gov/ed/collegescorecard/v1/schools'

In [3]:
"""colleges = [
    'Harvard',
]

for college_name in colleges:
    params = {
        'api_key': api_key,
        'school.name': college_name,
    }

response = requests.get(base_url, params=params)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f'API request failed with status code: {response.status_code}')
data"""

"colleges = [\n    'Harvard',\n]\n\nfor college_name in colleges:\n    params = {\n        'api_key': api_key,\n        'school.name': college_name,\n    }\n\nresponse = requests.get(base_url, params=params)\n\nif response.status_code == 200:\n    data = response.json()\n    print(data)\nelse:\n    print(f'API request failed with status code: {response.status_code}')\ndata"

In [4]:
# get list of college names
college_names = []
tables = pd.read_html('https://www.4icu.org/us/a-z/')
table_colleges = tables[0]

# flatten table
table_colleges.columns = table_colleges.columns.get_level_values(0)

# access the column names
table_colleges.columns = ['Rank', 'Name', 'Location']

# add each name to list
for college in table_colleges['Name']:
    college_names.append(college)

In [5]:
def calculate_mean_tuition(tuition_dict):
    """ calculates the average tution based on in state and out of state tuition

    Arguments:
        tuition_dict (dictionary): contains in state and out of state tuition costs

    Returns:
        tuition (int): final result of averaged tuition
        None: returns None if there are no entries for either in state or out of state tution
    TODO - why are some NaN and some are None"""

    # collect valid values
    valid_values = [value for value in [tuition_dict["in_state"], tuition_dict["out_of_state"]] if value is not None]

    if valid_values:
        # calculate average tuition
        tuition = sum(valid_values) / len(valid_values)
        return tuition
    else:
        return None

In [10]:
def calculate_diversity(div_dict):
    """TODO - docstring, not sure if this is the best way to do this
    calculates the proportion of students of color"""
    valid_values = [value for key, value in div_dict.items() if value is not None and key != "white" and key != "white_2000"]
    return sum(valid_values)

In [7]:
colleges = college_names[0:100]

# Initialize an empty DataFrame
result_df = pd.DataFrame(columns=['College', 'SAT', 'Earnings 6 yrs after entry', 
                                  'Share First Generation', 'Retention Rate', 'Student Faculty Ratio', 
                                  'Mean Tuition', 'Mean Net Price', 'Size', 'Diversity'])

for college_name in colleges:
    params = {
        'api_key': api_key,
        'school.name': college_name,
    }

    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        # Extract the desired data
        try:
            data = data['results'][0]['latest']
            sat_score = data['admissions']['sat_scores']['average']['overall']
            earnings = data['earnings']['6_yrs_after_entry']['median']
            first_gen_percentage = data['student']['share_firstgeneration']
            retention = data['student']['retention_rate']['overall']['full_time']
            student_faculty_ratio = data['student']['demographics']['student_faculty_ratio']
            mean_tuition = calculate_mean_tuition(data['cost']['tuition'])
            mean_net_price = data['cost']['avg_net_price']['overall'] #TODO - not sure if we should use average tuition or mean net price
            size = data['student']['size']
            poc_proportion = calculate_diversity(data["student"]["demographics"]["race_ethnicity"])
            
            # Create a DataFrame for the current college
            data_dict = {
                'College': [college_name],
                'SAT': [sat_score],
                'Earnings 6 yrs after entry': [earnings],
                'Share First Generation': [first_gen_percentage],
                'Retention Rate': [retention],
                'Student Faculty Ratio' : [student_faculty_ratio],
                'Mean Tuition': [mean_tuition],
                'Mean Net Price' : [mean_net_price],
                'Size': [size],
                'Diversity': [poc_proportion]
            }

            college_df = pd.DataFrame(data_dict)

            # Concatenate the college DataFrame to the result_df
            result_df = pd.concat([result_df, college_df], ignore_index=True)
        except (KeyError, IndexError):
            print(f"Data not found for {college_name}")
    else:
        print(f'API request failed with status code: {response.status_code} for {college_name}')

Data not found for A.T. Still University
API request failed with status code: 500 for Anderson University, Indiana
API request failed with status code: 500 for Aquinas College, Tennessee
Data not found for ArtCenter College of Design


,College,SAT,Earnings 6 yrs after entry,Share First Generation,Retention Rate,Student Faculty Ratio,Mean Tuition,Mean Net Price,Size,Diversity
0,Abilene Christian University,1186,45389,0.244523,0.7923,13,39350.0,29300,3131,NaN
1,Abraham Baldwin Agricultural College,1026,29825,0.344345,0.6515,19,7018.0,7203,3323,NaN
2,Academy of Art University,None,31825,0.408370,0.6899,14,25424.0,32736,5294,NaN
3,Adams State University,1040,36129,0.415131,0.5924,11,13806.5,14305,1484,NaN
4,Adelphi University,1200,60750,0.354207,0.8296,12,42475.0,28993,5168,NaN


In [11]:
result_df

,College,SAT,Earnings 6 yrs after entry,Share First Generation,Retention Rate,Student Faculty Ratio,Mean Tuition,Mean Net Price,Size,Diversity
0,Abilene Christian University,1186,45389,0.244523,0.7923,13,39350.0,29300,3131,NaN
1,Abraham Baldwin Agricultural College,1026,29825,0.344345,0.6515,19,7018.0,7203,3323,NaN
2,Academy of Art University,None,31825,0.408370,0.6899,14,25424.0,32736,5294,NaN
3,Adams State University,1040,36129,0.415131,0.5924,11,13806.5,14305,1484,NaN
4,Adelphi University,1200,60750,0.354207,0.8296,12,42475.0,28993,5168,NaN
...,...,...,...,...,...,...,...,...,...,...
91,Barclay College,None,29316,0.403101,0.5000,9,23850.0,23840,162,NaN
92,Bard College,None,30774,0.121711,0.2222,3,48800.0,21650,39,NaN
93,Bard College at Simon's Rock,None,30774,0.121711,0.7213,6,63583.0,33316,302,NaN
94,Barnard College,1484,66077,0.155650,0.9749,10,59687.0,32514,3007,NaN


In [9]:
# racial diversity
data["student"]["demographics"]["race_ethnicity"]

{'aian': 0.0024,
 'nhpi': 0.001,
 'asian': 0.0099,
 'black': 0.3878,
 'white': 0.1241,
 'unknown': 0.0119,
 'api_2000': None,
 'hispanic': 0.3803,
 'aian_2000': None,
 'black_2000': None,
 'white_2000': None,
 'two_or_more': 0.0246,
 'unknown_2000': None,
 'hispanic_2000': None,
 'aian_prior_2009': None,
 'black_non_hispanic': None,
 'non_resident_alien': 0.058,
 'white_non_hispanic': None,
 'hispanic_prior_2009': None,
 'asian_pacific_islander': None}